1. You have been given a partially implemented code for a feed-forward neural network using PyTorch. Your task is to complete the missing parts of the code to make it functional.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the neural network architecture
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x) # Complete this line to pass the output of the first layer through the activation function
        x = self.relu(x)# Complete this line to pass the output of the activation function through the second layer
        return x

# Define the hyperparameters
input_size = 10
hidden_size = 20
label_size = 5
learning_rate = 0.001
num_epochs = 1000

# Create the neural network object
model = NeuralNetwork(input_size, hidden_size, label_size)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# Generate some dummy data for training
train_data = torch.randn(100, input_size)
train_labels = torch.randint(label_size, (100,))

# Training loop
for epoch in range(num_epochs):
    # Forward pass
    # Complete this line to pass the training data through the model and obtain the predictions
    running_loss= 0
    last_loss= 0
    model.train(True)
    outputs = model(train_data)

    # Compute the loss
    loss = criterion(outputs, train_labels)
    running_loss += loss.item()
    
    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 100 == 0:
        print(f'Epoch: {epoch+1}/{num_epochs}, Loss: {loss.item()}')

# Test the trained model
test_data = torch.randn(10, input_size)
with torch.no_grad():
    # Complete this line to pass the test data through the model and obtain the predictions
    test_outputs = model(test_data)
    # Print the predictions
    _, predicted = torch.max(test_outputs.data, 1)
    print("Predictions:", predicted)
    

Epoch: 100/1000, Loss: 1.598299503326416
Epoch: 200/1000, Loss: 1.5951542854309082
Epoch: 300/1000, Loss: 1.5920203924179077
Epoch: 400/1000, Loss: 1.5886024236679077
Epoch: 500/1000, Loss: 1.5853784084320068
Epoch: 600/1000, Loss: 1.582106590270996
Epoch: 700/1000, Loss: 1.5788977146148682
Epoch: 800/1000, Loss: 1.5759257078170776
Epoch: 900/1000, Loss: 1.5731148719787598
Epoch: 1000/1000, Loss: 1.5705993175506592
Predictions: tensor([0, 3, 2, 0, 0, 0, 0, 2, 2, 2])


2. In this coding exercise, you need to implement the training of a deep MLP on the MNIST dataset using PyTorch and manually tune the hyperparameters. Follow the steps below to proceed:

* Load the MNIST dataset using torchvision.datasets.MNIST. The dataset contains handwritten digit images, and it can be easily accessed through PyTorch's torchvision module.

In [2]:
# Load the MNIST dataset
from torchvision import datasets, transforms
train_dataset = datasets.MNIST(root= "./data", train = True, download = True, transform = transforms)
test_dataset = datasets.MNIST(root = "./data", train = False, download = True, transform = transforms)


* Define your deep MLP model. Specify the number of hidden layers, the number of neurons in each layer, and the activation function to be used. You can use the nn.Sequential container to stack the layers.

In [8]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        layers = []
        # Input layer
        layers.append(nn.Linear(input_size, hidden_size[0]))
        layers.append(nn.ReLU())

        # Hidden layers
        for i in range(len(hidden_size) - 1):
            layers.append(nn.Linear(hidden_size[i], hidden_size[i + 1]))
            layers.append(nn.ReLU())

        # Output layer
        layers.append(nn.Linear(hidden_size[-1], output_size))

        # Create the model using nn.Sequential
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

* Set up the training loop and the hyperparameters. You can use the CrossEntropyLoss as the loss function and the Stochastic Gradient Descent (SGD) optimizer.

In [9]:
# Define hyperparameters
input_size = 784
hidden_sizes = [128, 128]
output_size = 10
batch_size = 64
learning_rate = 0.001
num_epochs = 10

# Create deep MLP model
model = MLP(input_size, hidden_sizes, output_size)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

* Train the model by iterating over the training dataset for the specified number of epochs. Compute the loss, perform backpropagation, and update the model's parameters. 

In [10]:
# Training loop
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode

    for batch_inputs, batch_labels in train_loader:
        # Flatten the input images
        batch_inputs = batch_inputs.view(-1, 784)

        # Forward pass
        outputs = model(batch_inputs)
        loss = criterion(outputs, batch_labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Print the loss after each epoch
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}')

TypeError: 'module' object is not callable

* Evaluate the trained model on the test dataset and calculate the accuracy (Please take a moment to consider the code below!)

In [18]:
# Evaluation

# Load the pre-trained model
model = MLP(input_size, hidden_sizes, output_size)
model.load_state_dict(torch.load('deep_mlp_model.pth'))
model.eval()

# Load the MNIST test dataset
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))]))
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# Evaluation
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images = images.view(-1, 784)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print("Accuracy: {:.2f}%".format(accuracy))


ValueError: too many values to unpack (expected 2)

* Manually tune the hyperparameters, such as the learning rate, by experimenting with different values and observing the performance. You can also search for the optimal learning rate by using techniques like learning rate range test, where you gradually increase the learning rate and monitor the loss.

In [ ]:
# TODO

3. In this coding exercise, you'll have an opportunity to explore the behavior of a deep neural network trained on the CIFAR10 image dataset. Follow the steps below:

* a. Construct a deep neural network (DNN) using 20 hidden layers, each comprising 100 neurons. To facilitate this exploration, employ the Swish activation function for each layer. Utilize nn.ModuleList to manage the layers effectively.

* b. Load the CIFAR10 dataset for training your network. Utilize the appropriate function, such as torchvision.datasets.CIFAR10. The dataset consists of 60,000 color images, with dimensions of 32×32 pixels. It is divided into 50,000 training samples and 10,000 testing samples. With 10 classes in the dataset, ensure that your network has a softmax output layer comprising 10 neurons. When modifying the model's architecture or hyperparameters, conduct a search to identify an appropriate learning rate. Implement early stopping during training and employ the Nadam optimization algorithm.

* c. Experiment by adding batch normalization to your network. Compare the learning curves obtained with and without batch normalization. Analyze whether the model converges faster with batch normalization and observe any improvements in its performance. Additionally, assess the impact of batch normalization on training speed.

* d. As an additional experiment, substitute batch normalization with SELU (Scaled Exponential Linear Units). Make the necessary adjustments to ensure the network self-normalizes. This involves standardizing the input features, initializing the network's weights using LeCun normal initialization (nn.init.kaiming_normal_), and ensuring that the DNN consists solely of dense layers. Observe the effects of utilizing SELU activation and self-normalization on the network's training stability and performance.

In [20]:
class Swish(nn.Module):
    def forward(self, x):
        return x * torch.sigmoid(x)


class DeepNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(DeepNN, self).__init__()

        layers = []
        for _ in range(20):
            layers.append(nn.Linear(input_size, hidden_size))
            layers.append(Swish())
            input_size = hidden_size

        layers.append(nn.Linear(hidden_size, output_size))

        self.layers = nn.ModuleList(layers)

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x


# Load the CIFAR10 dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Set up data loaders
batch_size = 64
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# Initialize the deep neural network
input_size = 32 * 32 * 3  # CIFAR10 image dimensions
hidden_size = 100
output_size = 10

model = DeepNN(input_size, hidden_size, output_size)

# Use Swish activation for all layers
model = model.to('cuda' if torch.cuda.is_available() else 'cpu')

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Nadam(model.parameters())

# Training loop with early stopping
num_epochs = 50
early_stop_counter = 5
best_loss = float('inf')
counter = 0

for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        images = images.view(-1, input_size).to('cuda' if torch.cuda.is_available() else 'cpu')
        labels = labels.to('cuda' if torch.cuda.is_available() else 'cpu')

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    with torch.no_grad():
        val_loss = 0
        for images, labels in test_loader:
            images = images.view(-1, input_size).to('cuda' if torch.cuda.is_available() else 'cpu')
            labels = labels.to('cuda' if torch.cuda.is_available() else 'cpu')

            val_outputs = model(images)
            val_loss += criterion(val_outputs, labels).item()

        val_loss /= len(test_loader)

        # Check for early stopping
        if val_loss < best_loss:
            best_loss = val_loss
            counter = 0
        else:
            counter += 1
            if counter >= early_stop_counter:
                print(f'Early stopping after {epoch + 1} epochs.')
                break

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item()}, Validation Loss: {val_loss}')


# Experiment with batch normalization
class DMLBN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(DMLBN, self).__init__()

        layers = []
        for _ in range(20):
            layers.append(nn.Linear(input_size, hidden_size))
            layers.append(nn.BatchNorm1d(hidden_size))
            layers.append(Swish())
            input_size = hidden_size

        layers.append(nn.Linear(hidden_size, output_size))

        self.layers = nn.ModuleList(layers)

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x


# Initialize the deep neural network with batch normalization
model_with_bn = DMLBN(input_size, hidden_size, output_size)
model_with_bn = model_with_bn.to('cuda' if torch.cuda.is_available() else 'cpu')

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer_with_bn = optim.Nadam(model_with_bn.parameters())

# Training loop with early stopping
best_loss_with_bn = float('inf')
counter_with_bn = 0

for epoch in range(num_epochs):
    model_with_bn.train()
    for images, labels in train_loader:
        images = images.view(-1, input_size).to('cuda' if torch.cuda.is_available() else 'cpu')
        labels = labels.to('cuda' if torch.cuda.is_available() else 'cpu')

        optimizer_with_bn.zero_grad()
        outputs = model_with_bn(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer_with_bn.step()

    # Validation
    model_with_bn.eval()
    with torch.no_grad():
        val_loss = 0
        for images, labels in test_loader:
            images = images.view(-1, input_size).to('cuda' if torch.cuda.is_available() else 'cpu')
            labels = labels.to('cuda' if torch.cuda.is_available() else 'cpu')

            val_outputs = model_with_bn(images)
            val_loss += criterion(val_outputs, labels).item()

        val_loss /= len(test_loader)

        # Check for early stopping
        if val_loss < best_loss_with_bn:
            best_loss_with_bn = val_loss
            counter_with_bn = 0
        else:
            counter_with_bn += 1
            if counter_with_bn >= early_stop_counter:
                print(f'Early stopping with batch normalization after {epoch + 1} epochs.')
                break

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item()}, Validation Loss: {val_loss}')


# Experiment with SELU activation and self-normalization
class DMLSELU(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(DMLSELU, self).__init__()

        layers = []
        for _ in range(20):
            layers.append(nn.Linear(input_size, hidden_size))
            layers.append(nn.SELU())
            input_size = hidden_size

        layers.append(nn.Linear(hidden_size, output_size))

        self.layers = nn.ModuleList(layers)

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x


# Initialize the deep neural network with SELU activation
model_with_selu = DMLSELU(input_size, hidden_size, output_size)
model_with_selu = model_with_selu.to('cuda' if torch.cuda.is_available() else 'cpu')


# Initialize weights with LeCun normal initialization
def init_weights(m):
    if isinstance(m, nn.Linear):
        init.kaiming_normal_(m.weight, a=0, mode='fan_in', nonlinearity='leaky_relu')


model_with_selu.apply(init_weights)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer_with_selu = optim.Nadam(model_with_selu.parameters())

# Training loop with early stopping
best_loss_with_selu = float('inf')
counter_with_selu = 0

for epoch in range(num_epochs):
    model_with_selu.train()
    for images, labels in train_loader:
        images = images.view(-1, input_size).to('cuda' if torch.cuda.is_available() else 'cpu')
        labels = labels.to('cuda' if torch.cuda.is_available() else 'cpu')

        optimizer_with_selu.zero_grad()
        outputs = model_with_selu(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer_with_selu.step()

    # Validation
    model_with_selu.eval()
    with torch.no_grad():
        val_loss = 0
        for images, labels in test_loader:
            images = images.view(-1, input_size).to('cuda' if torch.cuda.is_available() else 'cpu')
            labels = labels.to('cuda' if torch.cuda.is_available() else 'cpu')

            val_outputs = model_with_selu(images)
            val_loss += criterion(val_outputs, labels).item()

        val_loss /= len(test_loader)

        # Check for early stopping
        if val_loss < best_loss_with_selu:
            best_loss_with_selu = val_loss
            counter_with_selu = 0
        else:
            counter_with_selu += 1
            if counter_with_selu >= early_stop_counter:
                print(f'Early stopping with SELU after {epoch + 1} epochs.')
                break

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item()}, Validation Loss: {val_loss}')


Files already downloaded and verified
Files already downloaded and verified


AttributeError: module 'torch.optim' has no attribute 'Nadam'